In [34]:
print("Hello")

Hello


In [36]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch
# from transformers import LlamaForCausalLM, LlamaTokenizer

# model_name_or_path = "/home/c7361293/triz_llm/llama-2-7b-chat/model"
# pt_model_path = "/home/c7361293/triz_llm/llama-2-7b-chat/consolidated.00.pth"  # Path to your .pt or .pth file

# # tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
# # model = AutoModelForCausalLM.from_pretrained(model_name_or_path, state_dict=torch.load(pt_model_path))

# tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path)
# model_llm = LlamaForCausalLM.from_pretrained(model_name_or_path)
# # Example of generating text
# prompt = "What is a Machine? say in 20 words"
# inputs = tokenizer(prompt, return_tensors="pt")
# output_sequences = model_llm.generate(input_ids=inputs['input_ids'], max_length=50, num_return_sequences=1)

# generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
# print(generated_text)

Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.15it/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is a Machine? say in 20 words or less

A machine is a tool that performs a specific function or set of functions through the use of mechanical, electrical, electronic, or computer-controlled elements.


In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [3]:
import sys
import csv

csv.field_size_limit(sys.maxsize)

131072

In [4]:
loader = CSVLoader(file_path="./patents_with_samsung_2024_02_08.csv")

data = loader.load()

In [5]:
print(type(data))

<class 'list'>


In [6]:
print(type(data))


Document(page_content=': 351\nCountry: US\nPatentNumber: 20240041290\nApplicationReference: utility\nKind: A1\nDate: 2024-02-08\nAbstract: A docking station for a cleaner including a docking unit including an upper surface forming a travel route along a first direction for a robot cleaner to move upon to enter the docking station; and a floorcloth separator including a protrusion, and configured so that the protrusion protrudes from a predetermined area of the upper surface of the docking unit such that a free end of the protrusion is defined along a second direction different from the first direction, and while a robot cleaner with a floorcloth attached thereto travels on the travel route along the first direction, at least a portion of the protrusion is inserted between the robot cleaner and the attached floorcloth to separate the attached floorcloth from the robot cleaner.\nOrganizationName: SAMSUNG ELECTRONICS CO., LTD.\nSummaryOfInvention: {\'h-0001 - CROSS-REFERENCE TO RELATED AP

In [7]:
data[0].page_content

': 351\nCountry: US\nPatentNumber: 20240041290\nApplicationReference: utility\nKind: A1\nDate: 2024-02-08\nAbstract: A docking station for a cleaner including a docking unit including an upper surface forming a travel route along a first direction for a robot cleaner to move upon to enter the docking station; and a floorcloth separator including a protrusion, and configured so that the protrusion protrudes from a predetermined area of the upper surface of the docking unit such that a free end of the protrusion is defined along a second direction different from the first direction, and while a robot cleaner with a floorcloth attached thereto travels on the travel route along the first direction, at least a portion of the protrusion is inserted between the robot cleaner and the attached floorcloth to separate the attached floorcloth from the robot cleaner.\nOrganizationName: SAMSUNG ELECTRONICS CO., LTD.\nSummaryOfInvention: {\'h-0001 - CROSS-REFERENCE TO RELATED APPLICATIONS\': [\'This 

In [8]:
len(data)

299

In [26]:
# display(data)

In [9]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI, OpenAI

In [10]:
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings

In [ ]:
from langchain.embeddings import HuggingFace

In [11]:
 # Load and process the CSV data
loader = CSVLoader("./patents_with_samsung_2024_02_08.csv")
documents = loader.load()



# chroma_db = Chroma.from_documents(
#     documents, embeddings, persist_directory="./chroma_db"
# )
# chroma_db.persist()

In [17]:
import torch
torch.cuda.is_available()

False

In [18]:
nvidia-smi

NameError: name 'nvidia' is not defined

In [12]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [19]:
model_name = "BAAI/bge-large-en-v1.5"
# model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    # model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
    # ,
    # query_instruction="为这个句子生成表示以用于检索相关文章："
)
# model.query_instruction = "为这个句子生成表示以用于检索相关文章："

In [20]:
chroma_db = Chroma.from_documents(
    documents, model, persist_directory="./chroma_db"
)
chroma_db.persist()

In [22]:
vectordb = Chroma(persist_directory="./chroma_db", embedding_function= model)


In [52]:
retriever = vectordb.as_retriever(search_kwargs = {"k": 5})

In [51]:
retriever.search_type

'similarity'

In [53]:
retriever.search_kwargs

{'k': 5}

In [55]:
test_result = retriever.get_relevant_documents("docking station for a cleaner")

In [56]:
len(test_result)

5

In [59]:
test_result[1]

Document(page_content=': 354\nCountry: US\nPatentNumber: 20240041293\nApplicationReference: utility\nKind: A1\nDate: 2024-02-08\nAbstract: A docking station including a docking unit providing a traveling path for a robotic vacuum cleaner, having a mounted mopping cloth, that enters the docking unit; a mopping cloth separator disposed on the docking unit and configured to automatically separate the mounted mopping cloth from the robotic vacuum cleaner while the robotic vacuum cleaner is at a first position along the traveling path; and a mopping cloth coupler disposed on the docking unit and configured so that, after the mounted mopping cloth is separated from the robotic vacuum cleaner, and the robotic vacuum cleaner thereafter moves to a second position along the traveling path, the mopping cloth coupler supplies a mopping cloth to be mounted to the robotic vacuum cleaner from below the traveling path while the robotic vacuum cleaner is at the second position.\nOrganizationName: SAMSU

In [60]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from langchain.llms import R
# import torch

# class LocalLLM:
#     def __init__(self, model_path, tokenizer_path):
#         self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
#         self.model = AutoModelForCausalLM.from_pretrained(model_path)

#     def __call__(self, prompt):
#         inputs = self.tokenizer(prompt, return_tensors="pt")
#         outputs = self.model.generate(input_ids=inputs['input_ids'], max_length=50, num_return_sequences=1)
#         return self.tokenizer.decode(outputs[0], skip_special_tokens=True)
    
# #     model_name_or_path = ""
# # pt_model_path = "/home/c7361293/triz_llm/llama-2-7b-chat/consolidated.00.pth" 

# # Example usage
# model_path = "/home/c7361293/triz_llm/llama-2-7b-chat/model"
# tokenizer_path = "/home/c7361293/triz_llm/llama-2-7b-chat/model"
# local_llm = LocalLLM(model_path, tokenizer_path)

# # Now, you can pass this `local_llm` as the `llm` parameter
# qa_chain = RetrievalQA.from_chain_type(llm=local_llm, 
#                                        chain_type="stuff", 
#                                        retriever=retriever, 
#                                        return_source_documents=True)





In [39]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig


In [48]:
model_id = "/home/c7361293/triz_llm/llama-2-7b-chat/model"
 
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
                 model_id,
                 device_map="auto",
                #  load_in_8bit=True
                 )
pipe = pipeline("text-generation",
               model=model,
               tokenizer=tokenizer,
                max_new_tokens=1024
               )
 
llm = HuggingFacePipeline(pipeline=pipe)


Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.11it/s]


In [63]:
## Default LLaMA-2 prompt style
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template


In [64]:
sys_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

instruction = """CONTEXT:/n/n {context}/n

Question: {question}"""
get_prompt(instruction, sys_prompt)



"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. \n<</SYS>>\n\nCONTEXT:/n/n {context}/n\n\nQuestion: {question}[/INST]"

In [65]:
from langchain.prompts import PromptTemplate
prompt_template = get_prompt(instruction, sys_prompt)

llama_prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


In [66]:
chain_type_kwargs = {"prompt": llama_prompt}


In [67]:
from langchain.schema import prompt
# create the chain to answer questions
qa_chain1 = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type="stuff",
                                       retriever=retriever,
                                       chain_type_kwargs=chain_type_kwargs,
                                       return_source_documents=True)


In [69]:
# from langchain.chains import RetrievalQA
# from langchain import hub
 
# # Loads the latest version of the RAG prompt
# prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

# qa_chain = RetrievalQA.from_chain_type(
#     llm, retriever=retriever, chain_type_kwargs={"prompt": prompt}, chain_type="stuff"
# )


In [68]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])


In [70]:
# full example
query = "Give me patents with docking station for a cleaner"
llm_response = qa_chain1(query)
process_llm_response(llm_response)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


KeyboardInterrupt: 

In [50]:
question = "Give me top 5 patent id and summary with docking station for a cleaner"

result = qa_chain({"query": question})

print(result)



/home/c7361293/miniconda3/envs/triztest/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [ ]:
print(summary)

Summary:
The research aims to improve the energy efficiency of HVAC systems by developing a programmable thermostat that can control the HVAC unit remotely. The thermostat has a programmable schedule with separate heating and cooling setpoints, and the onboard controller of the HVAC unit can accept input signals from the thermostat. The research aims to optimize the energy consumption of the HVAC system by adjusting the temperature setpoints based on the programmable schedule. However, the research does not provide details on the specific energy efficiency improvements or the potential drawbacks of the proposed solution.


In [10]:
import os
import pandas as pd

from codecarbon import EmissionsTracker
import time

print("Start of code time: ",time.time())


# Path to the directory where your CSV files are stored
folder_path = './test_summarization_loop/'

# Loop through each file in the directory
for filename in os.listdir(folder_path):
    # Check if the file is a CSV
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)

        dynamic_name_prefix = filename[0:10]

        data_patent = pd.read_csv(folder_path + dynamic_name_prefix+'patents_with_samsung.csv')
        
        print("Start of code time of "+ dynamic_name_prefix +": ",time.time())

        tracker = EmissionsTracker()
        tracker.start()

        # Function to summarize text
        def summarize_abstract(text):

            torch.cuda.empty_cache()
            # prompt = f"Summarize this: {text} in 20 words"
            prompt = f"{text} \n Summarize the provided text focusing on the key improvement aimed at by the research, any potential drawbacks or degrading parameters mentioned, and any information on energy efficiency. The summary should clearly outline the main objectives, highlight any challenges or limitations if discussed, and note the presence or absence of energy efficiency details. Please keep the summary concise and output only the summary, ideally within five lines and start with 'Summary:'"
            # Check and set the padding token
            if tokenizer.pad_token is None:
                tokenizer.pad_token = tokenizer.eos_token  # Typically the EOS token can be used as pad token


            inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
            #  # Ensure that the attention mask and the pad token are set correctly
            # if 'attention_mask' not in inputs:
            #     inputs['attention_mask'] = torch.ones_like(inputs['input_ids'])
            # if 'pad_token_id' not in tokenizer.vocab:
            #     tokenizer.pad_token = tokenizer.eos_token

            inputs = inputs.to(device)
            
            output_sequences = model.generate(
                input_ids=inputs['input_ids'], 
                attention_mask=inputs['attention_mask'],
                max_length=5000, 
                num_return_sequences=1,
                do_sample=False  # Ensures more deterministic outputs
            )

            # print(tokenizer.decode(output_sequences, skip_special_tokens=True))
            text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
            summary_index = text.find("Summary:\n")
            summary = text
            if summary_index != -1:
                # Extract the text from "Begin Summary:" to the end of the string
                summary = text[summary_index:]

            return summary
            # Apply summarization to each abstract and store results in a new column
        summaries = []
        for claims in data_patent['Claims']:
            summary = summarize_abstract(abstract)
            summaries.append(summary)

        data_patent['SummaryofPatentClaims'] = summaries
        data_patent.to_csv('./summarized_patents/'+dynamic_name_prefix+'patents_with_samsung_summarized.csv', index=False)

        emissions: float = tracker.stop()
        print("Emissions for this code execution of "+dynamic_name_prefix+ "is: ", emissions)
        print("End of code time of "+dynamic_name_prefix+": ",time.time())


print("End of code time of "+dynamic_name_prefix+": ",time.time())


Start of code time:  1715648549.6975644
Start of code time of ipg240102_:  1715648549.7012768
Start of code time of ipg240109_:  1715648549.7036188
